In [ ]:
# default_exp FLIR_videocapture
# default_cls_lvl 3

## FLIR Video Capture
This class mimics openCV videocapture with three methods

In [ ]:
#hide
from nbdev.showdoc import *
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export
import imutils
import cv2
from imutils.video import FPS
import zmq
import numpy as np
import time
import skvideo.io
from FLIR_pubsub import FLIR_client_utils as flir

## FLIR Video Capture
This class mimics openCV videocapture with three methods

In [ ]:
# export
PORT = 5555
from threading import Thread
class FlirCapture:
    ''' Capture images from an networked FLIR camera across a tcp link. Mimics openCV VideoCapture class'''
    def __init__(self, src, name='FrontLeft', verbose=False):
        self.url = src
        self.name = name
        self._verbose = verbose
        self._isOpend = False
        self._socket_sub = None

        self._context = zmq.Context()

        # subscribe socket
        print(f"FlirCapture: Connecting to server {src}, {name}")
        try:
            self._socket_sub = self._context.socket(zmq.SUB)
            self._socket_sub.connect(f"tcp://{src}:{PORT}")
            self._socket_sub.setsockopt_string(zmq.SUBSCRIBE, name)
            self._isOpened = True
        except:
            print("Failed to open")
            self._isOpened = False



    def isOpened(self):
        ''' Returns bool True or False'''
        return self._isOpend

    def release(self):
        ''' end / stop'''
        self._stopped = True
        self._socket_sub.close()
        self._context.term()
        self._isOpend = False
        pass

    def read(self):
        '''read the last fetched buffer'''
        grabbed = False
        flir.poll_server(self.url, self.name)

        topic, rec_frame, md = flir.recv_frame(self._socket_sub)

        grabbed = True
        return grabbed, rec_frame

    # def _start(self):
    #     """start the thread to read frames from the video stream"""
    #     if self._verbose:
    #         print(f"[INFO] connecting to Cam: {self._src}")
    #
    #     self._stopped = False
    #     self._thread = Thread(target=self._update, name=self.name, args=())
    #     self._thread.daemon = True
    #     self._thread.start()
    #     return


if __name__ == '__main__':
    name1='FrontLeft'
    name2='FrontRight'
    src='localhost'
    stream1 = FlirCapture(src, name1)
    stream2 = FlirCapture(src, name2)
    print('Opened', stream1.isOpened())
    fps = FPS().start()

    while True:
        try:
            g, r = stream1.read()
            print(name1, g, r.shape)
            g, r = stream2.read()
            print(name2, g, r.shape)
            fps.update()
        except KeyboardInterrupt:
            break

    stream1.release()
    fps.stop()
    cv2.destroyAllWindows()
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

Connecting to server...
True (1800, 2400, 3)


In [ ]:
    # def _update(self):
    #     """keep looping infinitely until the thread is stopped"""
    #     while not self._stopped:
    #         topic, self.rec_frame, md = recv_frame(self._socket_sub)
    # 
    #         # 
	# 		# if self._stream is not None and self._stream.isOpened():
	# 		# 	(self.grabbed, self._frame) = self._stream.read()
	# 		# 	if self.grabbed:
	# 		# 		self._fps.update()
	# 		# 		self.last = datetime.datetime.now()
	# 		# 		time.sleep(0.01)
	# 		# else:
	# 		# 	self.connect()
	# 		# 	time.sleep(0.01)
	# 		# # 	time.sleep(1)
	# 		# if self._fps.elapsed() > 5:
	# 		# 	self._fps.stop()
	# 		# 	self.fps = self._fps.fps
	# 		# 	print(self.fps)
	# 		# 	if self._fps.numFrames == 0:
	# 		# 		self.connect()
    #         # 
	# 		# 	self._fps.start()
    # 
	# 	# Thread has stopped
	# 	if self._verbose:
	# 		print(f"[INFO] Stopped Cam: {self._src}")

In [ ]:
i = 1


